In [1]:
import algorithms.train as train
import algorithms.utils as utils
import algorithms.models as models
from enviroments import env_red, env_red_p2p, env_red_toy1, env_hubs1
from algorithms.agents import reinforce
import matplotlib.pyplot as plt
import numpy as np
import torch
import gym

In [2]:
env = env_hubs1(model="Exp", T=2000)
M = 33
F = 2

#Dimensiónes del espacio de estados
dims_state = list(env.C + 1)
dims_state.append(env.T+1)

#Dimensiones del espacio de acciones
dims_action = [M, F]


In [3]:
actor = models.PolicyNetwork(
    num_inputs=len(dims_state), 
    num_hiddens=[64, 64], 
    num_outputs=[M,F], 
    model="Softmax"
).double()

critc = models.ValueNetwork(
    num_inputs=len(dims_state), 
    num_hiddens=[64, 64], 
    num_outputs=1, 
).double()


In [4]:

agent = reinforce.ReinforceSoftmaxNN(actor, critc, gamma=.99, tau=.99, lr_actor= 1e-6)
Trainer = train.Trainer("sgd", "sgd")


In [6]:
s,_ = env.reset()

state = torch.as_tensor(s).double()
logits = actor(state).squeeze()
print(logits)
pi = torch.distributions.categorical.Categorical(logits=logits)
print(pi)
action = pi.sample()
print(f"La acción es: {action}, de longitud {len(action)}")

s,_ = env.reset()
action = agent.policy.act(s)
print(action[0])
print(action[1])
state = torch.as_tensor(s).double()
print(agent.policy.evaluate_logprob(state, action[0]))
print(agent.policy.evaluate_value(state))

s,_ = env.reset()
agent.select_action(s)
print(agent.buffer.actions)

tensor([[-0.0416, -0.0245],
        [ 0.0251,  0.0199],
        [ 0.0195,  0.0334],
        [ 0.0049,  0.0310],
        [-0.0631, -0.0165],
        [ 0.0368,  0.0637],
        [-0.0230,  0.0300],
        [-0.0375, -0.0238],
        [-0.0500, -0.0018],
        [ 0.0224,  0.0063],
        [-0.0225, -0.0110],
        [ 0.0154,  0.0033],
        [ 0.0394,  0.0063],
        [ 0.0040,  0.0156],
        [ 0.0274,  0.0211],
        [-0.0217, -0.0420],
        [ 0.0102, -0.0187],
        [-0.0078,  0.0215],
        [-0.0026, -0.0167],
        [-0.0149, -0.0484],
        [-0.0292, -0.0062],
        [ 0.0142,  0.0390],
        [ 0.0072,  0.0502],
        [-0.0019, -0.0319],
        [-0.0243, -0.0211],
        [ 0.0488,  0.0260],
        [ 0.0152,  0.0035],
        [ 0.0214, -0.0477],
        [ 0.0107, -0.0338],
        [-0.0332, -0.0161],
        [ 0.0030, -0.0383],
        [-0.0111, -0.0336],
        [-0.0150,  0.0315]], dtype=torch.float64, grad_fn=<SqueezeBackward0>)
Categorical(logits: torch.

In [15]:
states = torch.stack(agent.buffer.states, dim=0).detach()
print(states.shape)
actions = torch.stack(agent.buffer.actions, dim=0).detach().squeeze()
print(actions.shape)
logprobs = agent.policy.evaluate_logprob(states, actions)
print(logprobs.shape)

advantages = agent.update_critic()
print(advantages.shape)


torch.Size([4000, 10])
torch.Size([4000, 33])
torch.Size([4000, 33])
torch.Size([4000])


In [5]:
_, totals,_ = Trainer.train(env, agent, epochs=15000, max_steps=300, update_freq=4000, initial_offset=0)


plt.plot(totals)
plt.xlabel('Índice de la iteración')
plt.ylabel('Recompensa')
plt.title('Recompensa media')
plt.show()

KeyboardInterrupt: 

In [3]:
dimensions_actor = dims_state + dims_action
dimensions_critic = dims_state

freq = 5000

lr_actor = 1e-5

k = 5

In [4]:

actor = models.PolicyPARAFAC(dimensions_actor, k=k, model= "SoftMax", scale = 1)
critc = models.ValuePARAFAC(dimensions_critic, k=k, scale = 1)



agent = reinforce.ReinforceSoftmaxNN(actor, critc,discretizer_actor= None,discretizer_critic=None, gamma=.99, tau=.99, lr_actor=lr_actor)
Trainer = train.Trainer("sgd", "sgd")


In [ ]:
def forward(self, indices: np.ndarray) -> torch.Tensor:
        prod = torch.ones(self.k, dtype=torch.double)
        for i in range(len(indices)):
            idx = indices[i]
            factor = self.factors[i]
            prod *= factor[idx, :]
        if len(indices) < len(self.factors):
            res = []
            for cols in zip(
                *[self.factors[-(a + 1)].t() for a in reversed(range(self.nA))]
            ):
                kr = cols[0]
                for j in range(1, self.nA):
                    kr = torch.kron(kr, cols[j])
                res.append(kr)
            factors_action = torch.stack(res, dim=1)
            return torch.matmul(prod, factors_action.T)
        return torch.sum(prod, dim=-1)
tiene menú contextual

In [5]:
s, _ = env.reset()
agent.policy.act(s)

/home/jorejuela/Code/RMS/algorithms/models.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indices=torch.tensor(indices,dtype=torch.long).view(1, -1)


RuntimeError: Expect a 1D vector, but got shape [3, 5]

In [5]:

_, totals,_ = Trainer.train(env, agent, epochs=1, max_steps=1, update_freq=freq, initial_offset=0)


plt.plot(totals)
plt.xlabel('Índice de la iteración')
plt.ylabel('Recompensa')
plt.title('Recompensa media')
plt.show()

/home/jorejuela/Code/RMS/algorithms/models.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indices=torch.tensor(indices,dtype=torch.long).view(1, -1)


TypeError: expected Tensor as element 0 in argument 0, but got ParameterList